In [40]:
import pandas as pd

In [41]:
from google.colab import files
uploaded = files.upload()  # Upload your xlsx file here

Saving football_data.xlsx to football_data (1).xlsx


In [42]:
matches = pd.read_excel('football_data.xlsx')


In [54]:
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Venue_Code,Day_Code,Result_Code,Opponent_Code,Captain_Code,Formation_Code,Opp Formation_Code,Referee_Code,Match Report_Code,Team_Code
0,2024-08-17,12:30 (18:30),Premier League,Matchweek 1,Sat,Away,W,2,0,Ipswich Town,...,0,2,2,13,287,12,11,41,0,16
1,2024-08-25,16:30 (22:30),Premier League,Matchweek 2,Sun,Home,W,2,0,Brentford,...,1,3,2,3,287,12,17,38,0,16
2,2024-09-01,16:00 (22:00),Premier League,Matchweek 3,Sun,Away,W,3,0,Manchester Utd,...,0,3,2,19,287,12,11,3,0,16
3,2024-09-14,15:00 (21:00),Premier League,Matchweek 4,Sat,Home,L,0,1,Nott'ham Forest,...,1,2,1,23,287,12,11,23,0,16
4,2024-09-21,15:00 (21:00),Premier League,Matchweek 5,Sat,Home,W,3,0,Bournemouth,...,1,2,2,2,287,12,11,42,0,16


In [44]:
type(matches)

pandas.core.frame.DataFrame

In [62]:
matches.dtypes

,0
Date,datetime64[ns]
Time,object
Comp,object
Round,object
Day,object
Venue,object
Result,object
GF,int64
GA,int64
Opponent,object


In [67]:
# Auto-encode all object columns
for col in matches.select_dtypes(include='object').columns:
    if col not in ['Time', 'Date', 'Notes']:  # you can adjust this list
        matches[col + '_Code'] = matches[col].astype('category').cat.codes


matches['Date'] = matches['Date'].astype('category').cat.codes
# First, ensure Time is string
matches['Time'] = matches['Time'].astype(str)

# Extract local and UTC time from "12:30 (18:30)"
matches[['LocalTime', 'UTCTime']] = matches['Time'].str.extract(r'(\d{1,2}:\d{2})\s*\((\d{1,2}:\d{2})\)')

# Convert to datetime.time objects
matches['LocalTime'] = pd.to_datetime(matches['LocalTime'], format='%H:%M', errors='coerce').dt.time
matches['UTCTime'] = pd.to_datetime(matches['UTCTime'], format='%H:%M', errors='coerce').dt.time



In [68]:
# Pick features and target
feature_cols = ['Date', 'Comp_Code', 'Round_Code', 'Venue_Code', 'Day_Code', 'Opponent_Code',
                'Captain_Code', 'Formation_Code', 'Opp Formation_Code', 'Referee_Code', 'Match Report_Code',
                'Team_Code', 'LocalTime_Code', 'UTCTime_Code', 'GF', 'GA', 'xG', 'xGA', 'Poss', 'Sh',
                'SoT', 'Dist', 'FK', 'PK', 'npxG', 'npxG/Sh', 'G-xG']
X = matches[feature_cols]
y = matches['Result_Code']

# Check for any NaNs
X = X.dropna()
y = y[X.index]

In [69]:
X.head()

,Date,Comp_Code,Round_Code,Venue_Code,Day_Code,Opponent_Code,Captain_Code,Formation_Code,Opp Formation_Code,Referee_Code,...,xGA,Poss,Sh,SoT,Dist,FK,PK,npxG,npxG/Sh,G-xG
0,1030,0,0,0,2,13,287,12,11,41,...,0.5,62,18,5,14.8,0.0,0,2.6,0.15,-0.6
1,1034,0,11,1,3,3,287,12,17,38,...,0.5,62,19,8,13.6,1.0,0,2.5,0.14,-0.5
2,1036,0,22,0,3,19,287,12,11,3,...,1.4,47,11,3,13.4,0.0,0,1.8,0.16,1.2
3,1037,0,32,1,2,23,287,12,11,23,...,0.4,68,14,5,14.9,0.0,0,0.9,0.07,-0.9
4,1039,0,33,1,2,2,287,12,11,42,...,1.1,58,19,12,16.6,0.0,0,2.0,0.11,1.0


In [70]:
y.head()

,Result_Code
0,2
1,2
2,2
3,1
4,2


In [73]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

SEED = 23

train_X, test_X, train_y, test_y = train_test_split(X, y,
                                                    test_size = 0.25,
                                                    random_state = SEED)

gbc = GradientBoostingClassifier(n_estimators=300,
                                 learning_rate=0.05,
                                 random_state=100,
                                 max_features=5 )

gbc.fit(train_X, train_y)

pred_y = gbc.predict(test_X)

acc = accuracy_score(test_y, pred_y)
print("Gradient Boosting Classifier accuracy is : {:.2f}".format(acc))

Gradient Boosting Classifier accuracy is : 0.99
